In [1]:
import os
os.chdir('modelscope-agent/demo')


import sys
sys.path.append('../')
from modelscope_agent.llm import LLMFactory
from modelscope.hub.api import HubApi


model_name = 'modelscope-agent-7b'
model_cfg = {
    'modelscope-agent-7b':{
        'type': 'modelscope',
        'model_id': 'damo/ModelScope-Agent-7B',
        'model_revision': 'v1.0.0',
        'use_raw_generation_config': True,
        'custom_chat': True
    }
}


llm = LLMFactory.build_llm(model_name, model_cfg)

2023-09-27 16:28:20,416 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-09-27 16:28:20,418 - modelscope - INFO - TensorFlow version 2.13.0 Found.
2023-09-27 16:28:20,418 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2023-09-27 16:28:20,455 - modelscope - INFO - Loading done! Current index file version is 1.9.1, with md5 9fda7b6c6f6d961b96ab769dc9f618bd and a total number of 924 components indexed
2023-09-27 16:28:22,436 - modelscope - INFO - Use user-specified model revision: v1.0.0
The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".


In [20]:
import os
from modelscope.utils.config import Config
from modelscope_agent.llm import LLMFactory
from modelscope_agent.agent import AgentExecutor
from modelscope_agent.prompt import MSPromptGenerator

# get cfg from file, refer the example in config folder
model_cfg_file = os.getenv('MODEL_CONFIG_FILE', '../config/cfg_model_template.json')
model_cfg = Config.from_file(model_cfg_file)
tool_cfg_file = os.getenv('TOOL_CONFIG_FILE', '../config/cfg_tool_template.json')
tool_cfg = Config.from_file(tool_cfg_file)
os.environ['MODELSCOPE_API_TOKEN'] = "1b331ba1-37c1-4fb2-bff2-8c2a273731ec"

agent = AgentExecutor(llm, tool_cfg)

2023-09-27 15:43:07,448 - modelscope - INFO - Model revision not specified, use the latest revision: v1.1.0
2023-09-27 15:43:07,649 - modelscope - INFO - initiate model from /mnt/workspace/.cache/modelscope/damo/nlp_corom_sentence-embedding_chinese-base
2023-09-27 15:43:07,649 - modelscope - INFO - initiate model from location /mnt/workspace/.cache/modelscope/damo/nlp_corom_sentence-embedding_chinese-base.
2023-09-27 15:43:07,650 - modelscope - INFO - initialize model from /mnt/workspace/.cache/modelscope/damo/nlp_corom_sentence-embedding_chinese-base
2023-09-27 15:43:08,838 - modelscope - WARNING - No preprocessor field found in cfg.
2023-09-27 15:43:08,838 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2023-09-27 15:43:08,839 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/mnt/workspace/.cache/modelscope/damo/nlp_corom_sentence-embedding_chinese-ba

In [11]:
from modelscope_agent.tools import ModelscopePipelineTool
from modelscope.utils.constant import Tasks
from modelscope_agent.output_wrapper import AudioWrapper

class TexttoSpeechTool(ModelscopePipelineTool):
    default_model = 'damo/speech_sambert-hifigan_tts_zh-cn_16k'
    description = '文本转语音服务，将文字转换为自然而逼真的语音，可配置男声/女声'
    name = 'TexttoSpeech'
    parameters: list = [{
        'name': 'input',
        'description': '要转成语音的文本',
        'required': True
    }, {
        'name': 'gender',
        'description': '用户身份',
        'required': True
    }]
    task = Tasks.text_to_speech

    def _remote_parse_input(self, *args, **kwargs):
        if 'gender' not in kwargs:
            kwargs['gender'] = 'man'
        voice = 'zhibei_emo' if kwargs['gender'] == 'man' else 'zhiyan_emo'
        kwargs['parameters'] = voice
        kwargs.pop('gender')
        return kwargs

    def _parse_output(self, origin_result, remote=True):

        audio = origin_result['output_wav']
        return {'result': AudioWrapper(audio)}

In [14]:
from modelscope_agent.tools import ModelscopePipelineTool
from modelscope.utils.constant import Tasks

class TextAddressTool(ModelscopePipelineTool):
    default_model = 'damo/mgeo_geographic_elements_tagging_chinese_base'
    description = '地址解析服务，针对中文地址信息，识别出里面的元素，包括省、市、区、镇、社区、道路、路号、POI、楼栋号、户室号等'
    name = 'TextAddress'
    parameters: list = [{
        'name': 'input',
        'description': '用户输入的地址信息',
        'required': True
    }]
    task = Tasks.token_classification

    def _parse_output(self, origin_result, *args, **kwargs):
        final_result = {}
        for e in origin_result['output']:
            final_result[e['type']] = e['span']
        return final_result
    

In [15]:
from modelscope_agent.agent import AgentExecutor
additional_tool_list = {
    'TextAddress': TextAddressTool({}),
    'TexttoSpeech': TexttoSpeechTool({})
}


agent = AgentExecutor(llm, additional_tool_list=additional_tool_list, tool_retrieval=False)
available_tool_list = ['TextAddress','TexttoSpeech']
agent.set_available_tools(available_tool_list)

In [ ]:
agent.run('用女声读出来', remote=False)
